#Assessment 1: Mini RAG Pipeline with ChromaDB on Colab

Instructions:
- Use Google Colab for the task.
- Take 3–5 short documents (you can hardcode them).
- Preprocess the text by converting to lowercase and removing punctuation.
- Embed the documents using any method you prefer (e.g., sentence-transformers, HuggingFace, or other embedding techniques).
- Store the embeddings in ChromaDB or FAISS (use ChromaDB for simplicity).
- Accept a user query and retrieve the most relevant document based on cosine similarity.
- Generate a simple response by outputting the most relevant document.

In [1]:
# Installing Liabraries
!pip install sentence-transformers chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.7 MB/s eta 

In [2]:
# Take 3–5 short documents .
documents = ["Artificial Intelligence enables machines to think.",
             "Machine Learning is a subset of Artificial Intelligence. It learns from the patters of data.",
             "Deep Learning is a subset of Machine Learning.and it uses neural networks for learning."]

In [4]:
# Preprocessing
import re

clean_docs = []

for doc in documents:
    doc = doc.lower()
    doc = re.sub(r'[^\w\s]', '', doc)
    clean_docs.append(doc)


In [6]:
# Create Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = model.encode(clean_docs)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Store in Chromadb
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="mini_rag")

for i in range(len(doc_embeddings)):
    collection.add(
        ids=[str(i)],
        embeddings=[doc_embeddings[i]],
        documents=[clean_docs[i]]
    )


In [9]:
#  the query
query = "What is machine learning"
query = query.lower()
query = re.sub(r'[^\w\s]', '', query)

query_embedding = model.encode([query])


In [10]:
# Retrive most relevant document cosine similarity
result = collection.query(
    query_embeddings=query_embedding,
    n_results=1
)


In [11]:
# Output
print("Answer:", result["documents"][0][0])


Answer: machine learning is a subset of artificial intelligence it learns from the patters of data
